In [ ]:
!python -m pip install -e . >> /dev/null

In [ ]:
import toolviper
import xradio
import pathlib
import numba
import calviper

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from xradio import measurement_set as ms

ModuleNotFoundError: No module named 'calviper'

In [ ]:
if not pathlib.Path.cwd().joinpath("data/gaincaltest2.ps.zarr").exists():
    toolviper.utils.data.download("gaincal.test.zarr", "data")

In [ ]:
ps = ms.open_processing_set("data/gaincaltest2.ps.zarr")

sub_ps = ps.sel(intents="CALIBRATE_DELAY#ON_SOURCE", scan_number=2)
sub_ps.summary()

In [ ]:
dataset = sub_ps["gaincaltest2_0"]

In [ ]:
V = dataset.VISIBILITY.mean(dim=["time", "frequency"], keepdims=True, keep_attrs=True).data.compute()
V.shape

In [ ]:
cm = calviper.factory.jones.CalibrationMatrix()

In [ ]:
G = cm.create_jones("gain").empty_like(dataset)

In [ ]:
G.gain.initialize()

In [ ]:
G

In [ ]:
G.PARAMETER.shape

In [ ]:
G.coords.items()

In [ ]:
v = V

full_antenna_list = np.union1d(
    dataset.baseline_antenna1_name.to_numpy(),
    dataset.baseline_antenna2_name.to_numpy()
)

full_antenna_list

encoder, antennas = calviper.math.tools.encode(full_antenna_list)

index_a = encoder.transform(dataset.baseline_antenna1_name.to_numpy())
index_b = encoder.transform(dataset.baseline_antenna2_name.to_numpy())

v_ = calviper.math.tools.build_visibility_matrix(array=v, index_a=index_a, index_b=index_b)

In [ ]:
v_.shape

In [ ]:
solver = calviper.math.solver.least_squares.LeastSquaresSolver()

In [ ]:
gain_solutions = solver.solve(
    vis=v_,
    iterations=50,
    optimizer=calviper.math.optimizer.MeanSquaredError(alpha=0.25),
    stopping=1e-4
)

In [ ]:
t = np.linspace(1, len(solver.losses), len(solver.losses))

plt.scatter(solver.losses, t)

In [ ]:
# Gain solutions

print(f"X: {solver.parameter[..., 0, :]}")
print(f"Y: {solver.parameter[..., 1, :]}")

In [ ]:
solver.parameter.shape